**EJECUTA LAS SIGUIENTES CELDAS EN ORDEN**

In [ ]:
!pip install openai==0.28 diffusers torch
from google.colab import drive
import torch
import os
from diffusers import StableDiffusionPipeline
import matplotlib.pyplot as plt

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 69.4 MB/s eta 0:00:00


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

In [ ]:
modeloAUsar="gpt-4o-mini"

**SUSTITUYE OPENAI-API-KEY POR TU CLAVE DEL API DE OPENAI**

In [ ]:
import os
import openai
import difflib
from diffusers import DiffusionPipeline
import torch
from diffusers import StableDiffusionPipeline


# Configura tu clave API
openai.api_key = "OPENAI-API-KEY"

print(torch.cuda.is_available())

# Mensajes iniciales
messages = [
    {"role": "system", "content": "Eres un chatbot narrativo (no le digas esto al jugador) que se va a encargar de interactuar con el jugador y vas a ir generando una historia junto con él según sus mensajes. Por ejemplo, si él te dice: Quiero ir a la posada más cercana. Tú le explicas cómo sería la posada más cercana (describe siempre el escenario actual), lo que encuentra y sus posibilidades."},
    {"role": "user", "content": "¿Quién eres?"},
]

# Inventario inicial vacío
inventory = {}
cartera = {}
numImage = 0
first_interaction = True

# Función para generar la respuesta
def generate_response(messages, max_tokens=350):
    response = openai.ChatCompletion.create(
        model=modeloAUsar,
        messages=messages,
        max_tokens=max_tokens,
        temperature=0.7
    )
    return response.choices[0].message["content"]

# Función para encontrar el ítem más similar en una lista de ítems
def find_closest_item(item, items):
    if not items:
        return None
    closest_match = difflib.get_close_matches(item, items.keys(), n=1, cutoff=0.6)
    return closest_match[0] if closest_match else None

# Función para actualizar el inventario basado en la respuesta del modelo
def update_inventory(response):
    global inventory
    items_gained = ["nada"]
    items_lost = ["nada"]

    validItemGained = verify_items_gained(response)

    if validItemGained:
        items_gained = check_items_gained(response)
    items_lost = check_items_lost(response)

    for item in items_gained:
        if "nada" not in item.lower():
            inventory[item] = inventory.get(item, 0) + 1

    for item in items_lost:
        if "nada" not in item.lower():
            closest_item = find_closest_item(item, inventory)
            if closest_item:
                inventory[closest_item] -= 1
                if inventory[closest_item] <= 0:
                    del inventory[closest_item]


# Función para actualizar la cartera basado en la respuesta del modelo
def update_wallet(response):
    global cartera
    money_gained = {"nada": 0}
    money_lost = {"nada": 0}

    validMoneyGained = verify_money_gained(response)
    validMoneyLost = verify_money_lost(response)

    if validMoneyGained:
        money_gained = check_money_gained(response)
    if validMoneyLost:
        money_lost = check_money_lost(response)

    for currency, amount in money_gained.items():
        if currency != "nada":
            closest_currency = find_closest_item(currency, cartera)
            if closest_currency:
                cartera[closest_currency] += amount
            else:
                cartera[currency] = amount

    for currency, amount in money_lost.items():
        if currency != "nada":
            closest_currency = find_closest_item(currency, cartera)
            if closest_currency and cartera[closest_currency] >= amount:
                cartera[closest_currency] -= amount
                if cartera[closest_currency] <= 0:
                    del cartera[closest_currency]
            else:
                print(f"No tienes suficientes {currency} para realizar esta acción.")
                return False  # Indica que no se pudo realizar la transacción
    return True  # Indica que la transacción fue exitosa

#Función para verificar si se ha conseguido un item
def verify_items_gained(response):
    verification_prompt = (
        f"El siguiente  texto corresponde a la respuesta del sistema en una historia narrativa, analízalo y responde  lo que se pregunta a continuación: \n"
        f"Texto: {response}\n"
        f"¿Ha ganado el jugador algún objeto explícitamente en este evento? Estudia con cuidado el texto y comprueba que  el jugador haya adquirido el objeto de forma explícita, una simple mención a un objeto no cuenta como una adquisición Responde con 'sí' o 'No'.\n"
        f"La posibilidad de adquirir items(armas, pociones, o  lo que sea) no cuenta como una adquisición válida entonces la respuesta sería no, ya que la adquisición no es explícita, es solo una posibilidad\n"
        f"En cambio asegurate de que si es explícito que se recoje o consigue, o te dan,  o te entregan(o cualquier sinónimo) un objeto SEA CUAL SEA, por  ejemplo  <cojo el  móvil> también cuenta como que has obtenido un móvil, ABSOLUTAMENTE CUALQUIER OBJETO QUE PASES A  LLEVAR ENCIMA DE FORMA EXPLÍCITA, la  respuesta sea SI, por ejemplo, se recoge un palo del suelo, compras un arma, fabricas una poción. \n"

    )
    verification_messages = [
        {"role": "system", "content": "Eres un verificador que identifica si el jugador ha ganado objetos en el texto proporcionado.  Ten en  cuenta que una mención a un objeto en el texto no implica una ganancia, revísalo bien para asegurarte de que  sea una ganancia explícita en el acto."},
        {"role": "user", "content": verification_prompt}
    ]
    response = openai.ChatCompletion.create(
        model=modeloAUsar,
        messages=verification_messages,
        max_tokens=256,
        temperature=0.7
    )
    verification_text = response.choices[0].message["content"]
    #print(verification_text)
    return "sí" in verification_text.lower() or "si" in verification_text.lower()
#Función para verificar si se ha perdido un item
def verify_items_lost(response):
    verification_prompt = (
        f"El siguiente  texto corresponde a la respuesta del sistema en una historia narrativa, analízalo y responde  lo que se pregunta a continuación: \n"
        f"Texto: {response}\n"
        f"¿Ha perdido el jugador algún objeto explícitamente en este evento? Estudia con cuidado el texto y comprueba que  el jugador haya perdido el objeto de forma explícita, una simple mención a un objeto no cuenta como una pérdida. Responde con 'sí' o 'No'.\n"
        f"La posibilidad de perder items(armas, pociones, o  lo que sea, cualquier tipo  de objeto) no cuenta como una pérdida válida entonces la respuesta sería no, ya que la pérdida no es explícita, es solo una posibilidad\n"
        f"En cambio asegurate de que si es explícito que se consume o rompe un objeto la  respuesta sea sí, por ejemplo, rompes el palo, vendes un arma, gastas o utiliza una poción. \n"
    )
    verification_messages = [
        {"role": "system", "content": "Eres un verificador que identifica si el jugador ha perdido objetos en el texto proporcionado. Ten en  cuenta que una mención a un objeto en el texto no implica una pérdida, revísalo bien para asegurarte de que  sea una pérdida explícita en el acto."},
        {"role": "user", "content": verification_prompt}
    ]
    response = openai.ChatCompletion.create(
        model=modeloAUsar,
        messages=verification_messages,
        max_tokens=256,
        temperature=0.7
    )
    verification_text = response.choices[0].message["content"]
    #print(verification_text)
    return "sí" in verification_text.lower() or "si" in verification_text.lower()

#Función para verificar si se ha ganado dinero
def verify_money_gained(response):
    verification_prompt = (
        f"El siguiente  texto corresponde a la respuesta del sistema en una historia narrativa, analízalo y responde  lo que se pregunta a continuación: \n"
        f"Texto: {response}\n"
        f"¿Ha ganado el jugador dinero de forma explícita(oro, plata, euros, cualquier tipo de dinero) explícitamente en este evento? Estudia con cuidado el texto y comprueba que  el jugador haya adquirido dinero de forma explícita, una simple mención a una moneda o a dinero no cuenta como una adquisición del mismo. Responde con 'sí' o 'No'.\n"
        f"La posibilidad de ganar dinero(oro, plata, euros, o  lo que sea) no cuenta como una ganancia válida entonces la respuesta sería NO, ya que la ganancia no es explícita, es solo una posibilidad, por ejemplo, aunque aparezcan 50  monedas de oro, si no se especifica que estas monedas se consiguen, encuentran o ganan, la respuesta sería no\n"

            )
    verification_messages = [
        {"role": "system", "content": "Eres un verificador que identifica si el jugador ha ganado dinero en el texto proporcionado.Ten en  cuenta que una mención al dinero en el texto no implica una ganancia, revísalo bien para asegurarte de que  sea una ganancia explícita en el acto."},
        {"role": "user", "content": verification_prompt}
    ]
    response = openai.ChatCompletion.create(
        model=modeloAUsar,
        messages=verification_messages,
        max_tokens=256,
        temperature=0.7
    )
    verification_text = response.choices[0].message["content"]
    #print(verification_text)
    return "sí" in verification_text.lower() or "si" in verification_text.lower()

#Función para verificar si se ha perdido dinero
def verify_money_lost(response):
    verification_prompt = (
        f"El siguiente  texto corresponde a la respuesta del sistema en una historia narrativa, analízalo y responde  lo que se pregunta a continuación: \n"
        f"Texto: {response}\n"
        f"¿Ha perdido el jugador dinero de forma explícita(oro, plata, euros, cualquier tipo de dinero) explícitamente en este evento? Estudia con cuidado el texto y comprueba que  el jugador haya perdido dinero de forma explícita, una simple mención a una moneda o a dinero no cuenta como una pérdida del mismo. Responde con 'sí' o 'No'.\n"
        f"La posibilidad de perder dinero(oro, plata, euros, o  lo que sea) no cuenta como una pérdida válida entonces la respuesta sería no, ya que la pérdida no es explícita, es solo una posibilidad\n"
            )
    verification_messages = [
        {"role": "system", "content": "Eres un verificador que identifica si el jugador ha perdido  dinero en el texto proporcionado. Ten en  cuenta que una mención al dinero en el texto no implica una pérdida, revísalo bien para asegurarte de que  sea una pérdida explícita en el acto."},
        {"role": "user", "content": verification_prompt}
    ]
    response = openai.ChatCompletion.create(
        model=modeloAUsar,
        messages=verification_messages,
        max_tokens=256,
        temperature=0.7
    )
    verification_text = response.choices[0].message["content"]
    #print(verification_text)
    return "sí" in verification_text.lower() or "si" in verification_text.lower()

#Funcion para ver que  items se han ganado
def check_items_gained(response):
    validation_prompt = (
        f"Texto: {response}\n"
        "Ejemplos:\n"
        "Texto: 'Encuentras una espada mágica y un escudo antiguo en el cofre.' Respuesta: 'Obtienes: espada mágica, escudo antiguo.'\n"
        "Texto: 'El mago te entrega una poción curativa.' Respuesta: 'Obtienes: poción curativa.'\n"
        "Texto: 'Fabricas un aceite para embadurnar la espada.' Respuesta: 'Obtienes: Aceite para embadurnar la espada.'\n"
        "Texto: 'No encuentras nada de valor en la habitación.' Respuesta: 'Obtienes: nada.'\n"
        "Texto: 'Los guardias no te dan ningún objeto.' Respuesta: 'Obtienes: nada.'\n"
        "Texto: 'El mercader te vende una armadura de cuero.' Respuesta: 'Obtienes: armadura de cuero.'\n"
        "Texto: 'La Espada del Fénix tiene un precio justo y el herrero te asegura que es una inversión que valdrá la pena en tus futuras aventuras. ¿Decides adquirirla para convertirla en tu fiel compañera en tus hazañas por Azinoz?' Respuesta: 'Obtienes: nada.'\n"
        "Texto: 'El mercader te muestra varios escudos y sus precios.' Respuesta: 'Obtienes: nada.'\n"
        "Texto: '¿Qué espada te interesa adquirir, noble caballero Susi?' Respuesta: 'Obtienes: nada.'\n"
        "¿Qué objetos físicos ha ganado el jugador explícitamente en este evento? "
        "Solo considera los objetos que el jugador ha adquirido de forma explícita, no menciones objetos que solo están en intención o posibilidad. "
        "El dinero, oro, plata, o cualquier moneda no se considera como un objeto que se pueda añadir al inventario."
        "Tampoco cuentan como obtenidos objetos que han sido ofrecidos al jugador por ejemplo en una tienda pero que este no ha adquirido explícitamente todavía. "
        "Responde en una lista con el formato 'Obtienes: <objeto>'. "
        "Si no ha ganado nada, responde 'Obtienes: nada'. "
        "Responde siempre 'nada', no respondas con ninguna otra variación ni nada por el estilo, simplemente eso, NADA, gracias."
    )
    validation_messages = [
        {"role": "system", "content": "Eres un conversor que identifica objetos físicos explícitamente obtenidos por el jugador en el texto proporcionado."},
        {"role": "user", "content": validation_prompt}
    ]
    response = openai.ChatCompletion.create(
        model=modeloAUsar,
        messages=validation_messages,
        max_tokens=256,
        temperature=0.7
    )
    gained_items_text = response.choices[0].message["content"]
    #print("Objetos obtenidos:", gained_items_text)
    items = [line.split(": ")[1].strip() for line in gained_items_text.split('\n') if line.startswith("Obtienes:")]
    if not items:
        items.append("nada")
    items = [item.rstrip('.') for item in items]
    return items
#Funcion para ver que  items se han perdido
def check_items_lost(response):
    validation_prompt = (
        f"Texto: {response}\n"
        "Ejemplos:\n"
        "Texto: 'Consumes una poción curativa durante el combate.' Respuesta: 'Pierdes: poción curativa.'\n"
        "Texto: 'Devuelves la espada al herrero.' Respuesta: 'Pierdes: espada.'\n"
        "Texto: 'No pierdes ningún objeto durante esta interacción.' Respuesta: 'Pierdes: nada.'\n"
        "Texto: 'Utilizas la poción para curarte.' Respuesta: 'Pierdes: poción.'\n"
        "Texto: 'Te comes las manzanas.' Respuesta: 'Pierdes: manzanas.'\n"
        "Texto: 'Has perdido el escudo en la batalla.' Respuesta: 'Pierdes: escudo.'\n"
        "Texto: 'No has perdido ningún objeto en este evento.' Respuesta: 'Pierdes: nada.'\n"
        "¿Qué objetos físicos ha perdido, consumido o devuelto explícitamente el jugador en este evento? "
        "Solo considera los objetos que el jugador ha perdido, consumido o devuelto de forma explícita, no menciones objetos que solo están en intención o posibilidad. "
        "Responde en una lista con el formato 'Pierdes: <objeto>'. "
        "Si no ha perdido nada, responde 'Pierdes: nada'."
    )
    validation_messages = [
        {"role": "system", "content": "Eres un conversor que identifica objetos físicos explícitamente perdidos, consumidos o devueltos por el jugador en el texto proporcionado."},
        {"role": "user", "content": validation_prompt}
    ]
    response = openai.ChatCompletion.create(
        model=modeloAUsar,
        messages=validation_messages,
        max_tokens=256,
        temperature=0.7
    )
    lost_items_text = response.choices[0].message["content"]
    #print("Objetos perdidos:", lost_items_text)
    items = [line.split(": ")[1].strip() for line in lost_items_text.split('\n') if line.startswith("Pierdes:")]
    if not items:
        items.append("nada")
    items = [item.rstrip('.') for item in items]
    return items
#Funcion para ver que tipo y cuanto dinero se ha ganado
def check_money_gained(response):
    validation_prompt = (
        f"Texto: {response}\n"
        "Ejemplos:\n"
        "Texto: 'El mercader te paga 50 monedas de oro por tus servicios.' Respuesta: 'Ganas: 50 monedas de oro.'\n"
        "Texto: 'Encuentras una bolsa con 20 piezas de plata en el suelo.' Respuesta: 'Ganas: 20 piezas de plata.'\n"
        "Texto: 'No encuentras ninguna moneda en la habitación.' Respuesta: 'Ganas: nada.'\n"
        "Texto: 'Recibes una recompensa de 100 monedas de cobre por tu valentía.' Respuesta: 'Ganas: 100 monedas de cobre.'\n"
        "Texto: 'Compras una espada por 10 monedas de oro.' Respuesta: 'Ganas: nada.'\n"
        "Texto: 'Donas 15 monedas de oro al pobre.' Respuesta: 'Ganas: nada.'\n"
        "Texto: 'El mercader te muestra varios escudos y sus precios.' Respuesta: 'Ganas: nada.'\n"
        "Texto: 'Pagas 50 monedas de oro al mercader.' Respuesta: 'Ganas: nada.'\n"
        "Texto: 'Pierdes 30 monedas de plata en una apuesta.' Respuesta: 'Ganas: nada.'\n"
        "¿Qué cantidad de dinero ha ganado el jugador explícitamente en este evento? "
        "Solo considera el dinero que el jugador ha adquirido de forma explícita, no menciones dinero que solo está en intención o posibilidad. "
        "No cuentes transacciones donde el jugador ha gastado o donado dinero o ha perdido dinero. "
        "Responde en una lista con el formato 'Ganas: <cantidad> <moneda>'. "
        "Si no ha ganado nada, responde 'Ganas: nada'."
    )
    validation_messages = [
        {"role": "system", "content": "Eres un conversor que identifica dinero explícitamente ganado por el jugador en el texto proporcionado."},
        {"role": "user", "content": validation_prompt}
    ]
    response = openai.ChatCompletion.create(
        model=modeloAUsar,
        messages=validation_messages,
        max_tokens=256,
        temperature=0.7
    )
    gained_money_text = response.choices[0].message["content"]
    #print("Dinero ganado:", gained_money_text)
    money = {}
    for line in gained_money_text.split('\n'):
        if line.startswith("Ganas:"):
            parts = line.split(": ")[1].strip().split()
            if parts[0].isdigit():
                amount = int(parts[0])
                currency = " ".join(parts[1:])
                money[currency] = money.get(currency, 0) + amount
    if not money:
        money["nada"] = 0
    return money

#Funcion para ver que tipo y cuanto dinero se ha perdido
def check_money_lost(response):
    validation_prompt = (
        f"Texto: {response}\n"
        "Ejemplos:\n"
        "Texto: 'Pagas 30 monedas de oro al mercader por la espada.' Respuesta: 'Pierdes: 30 monedas de oro.'\n"
        "Texto: 'Pierdes 15 piezas de plata en una apuesta.' Respuesta: 'Pierdes: 15 piezas de plata.'\n"
        "Texto: 'No pierdes ninguna moneda durante esta interacción.' Respuesta: 'Pierdes: nada.'\n"
        "Texto: 'Donas 10 monedas de cobre a los pobres.' Respuesta: 'Pierdes: 10 monedas de cobre.'\n"
        "Texto: 'Das 15 euros a un mendigo de la ciudad.' Respuesta: 'Pierdes: 15 euros.'\n"
        "Texto: 'Das 15 monedas de oro al pobre.' Respuesta: 'Pierdes: 15 monedas de oro.'\n"
        "Texto: 'El mercader te muestra varios escudos y sus precios.' Respuesta: 'Pierdes: nada.'\n"
        "¿Qué cantidad de dinero ha perdido el jugador explícitamente en este evento? "
        "Solo considera el dinero que el jugador ha perdido de forma explícita, no menciones dinero que solo está en intención o posibilidad. "
        "No cuentes transacciones donde el jugador ha recibido dinero. "
        "Responde en una lista con el formato 'Pierdes: <cantidad> <moneda>'. "
        "Si no ha perdido nada, responde 'Pierdes: nada'."
    )
    validation_messages = [
        {"role": "system", "content": "Eres un conversor que identifica dinero explícitamente perdido por el jugador en el texto proporcionado."},
        {"role": "user", "content": validation_prompt}
    ]
    response = openai.ChatCompletion.create(
        model=modeloAUsar,
        messages=validation_messages,
        max_tokens=256,
        temperature=0.7
    )
    lost_money_text = response.choices[0].message["content"]
    #print("Dinero perdido:", lost_money_text)
    money = {}
    for line in lost_money_text.split('\n'):
        if line.startswith("Pierdes:"):
            parts = line.split(": ")[1].strip().split()
            if parts[0].isdigit():
                amount = int(parts[0])
                currency = " ".join(parts[1:])
                money[currency] = money.get(currency, 0) + amount
    if not money:
        money["nada"] = 0
    return money
#Funcion para comprobar si hay  dinero suficiente para realizar una acción
def validate_money(action, context, cartera, max_tokens=5):
    validation_prompt = (
        f"Contexto: {context}\n"
        f"Cartera: {cartera}\n"
        f"Acción del jugador: {action}\n"
        f"Ocupate solo de lo relacionado con el dinero, si no tiene relación  con el ámbito económico o monetario de la narración igóra la acción y valídala DICIENDO sí, recuerda, si validas una acción di  solo <sí>"
        "¿Tiene sentido esta acción en el contexto de la cartera del jugador (El jugador solo tiene el dinero que hay en su cartera)? "
        "Si intenta comprar algo pero no tiene dinero en la cartera, no sería una acción válida.\n"
        "Ten en cuenta que las acciones son en interacción con un bot narrativo entonces si la acción es del estilo de <Soy el caballero Jesús y quiero  vivir una aventura  en el mundo fantástico de Elanor>, entonces tendrás que dar la acción por  válida diciendo si ."
        "Responde SOLO con 'sí' o 'no'.\n"
    )
    validation_messages = [
        {"role": "system", "content": "Eres un validador de acciones en una historia interactiva para verificar el dinero."},
        {"role": "user", "content": validation_prompt}
    ]
    response = openai.ChatCompletion.create(
        model=modeloAUsar,
        messages=validation_messages,
        max_tokens=256,
        temperature=0.7
    )
    validation_response = response.choices[0].message["content"]
    #print(f"\nComprobador de dinero: {validation_response}")
    return "sí" in validation_response.lower() or "si" in validation_response.lower()

#Funcion para comprobar si tienes los items necesarios para realizar una acción.
def validate_items(action, context, inventory, max_tokens=5):
    validation_prompt = (
        f"Contexto: {context}\n"
        f"Inventario: {inventory}\n"
        f"Acción del jugador: {action}\n"
        "¿Tiene sentido esta acción en el contexto del inventario del jugador (El jugador solo tiene los ítems que hay en su inventario)? "
        "Si intenta realizar una acción que requiere un ítem que no tiene, no sería una acción válida.\n"
        "Responde con 'sí' o 'no'.\n"
        "Ten en cuenta que las acciones son en interacción con un bot narrativo entonces si la acción es del estilo de <Soy el caballero Jesús y quiero  vivir una aventura  en el mundo fantástico de Elanor>, entonces tendrás que dar la acción por  válida diciendo si."
    )
    validation_messages = [
        {"role": "system", "content": "Eres un validador de acciones en una historia interactiva para verificar los ítems."},
        {"role": "user", "content": validation_prompt}
    ]
    response = openai.ChatCompletion.create(
        model=modeloAUsar,
        messages=validation_messages,
        max_tokens=256,
        temperature=0.7
    )
    validation_response = response.choices[0].message["content"]
    #print(f"\nComprobador de ítems: {validation_response}")
    return "sí" in validation_response.lower() or "si" in validation_response.lower()  # Incluye "si" para no tener problemas con acentos


#Funcion para comprobar si la acción tiene  sentido contextual
def validate_contextual(action, context, max_tokens=5):
    validation_prompt = (
        f"Contexto: {context}\n"
        f"Acción del jugador: {action}\n"
        "¿Tiene sentido esta acción en el contexto del mundo de la historia? "
        "Por ejemplo, no puedes sacar una pistola si la aventura es en un mundo de fantasía medieval.\n"
        "Responde con 'sí' o 'no'."
    )
    validation_messages = [
        {"role": "system", "content": "Eres un validador de acciones en una historia interactiva para verificar el contexto del mundo."},
        {"role": "user", "content": validation_prompt}
    ]
    response = openai.ChatCompletion.create(
        model=modeloAUsar,
        messages=validation_messages,
        max_tokens=256,
        temperature=0.7
    )
    validation_response = response.choices[0].message["content"]
    #print(f"\nComprobador contextual: {validation_response}")
    return "sí" in validation_response.lower() or "si" in validation_response.lower()

# Función para validar las acciones del jugador
def validate_action(action, context, inventory, cartera):
    valid_money = validate_money(action, context, cartera)
    valid_items = validate_items(action, context, inventory)
    validate_context =  validate_contextual(action, context)
    return valid_money and valid_items and validate_context

#Función para obtener el texto que se usará para la generación de la imagen
def textToImageGeneration(context):
    validation_prompt = (
        f"La última respuesta del chatbot narrativo es la siguiente: {context}\n"
        "¿Podrías generar una descripción muy corta basada en el contexto. El estilo de la descripción debe  ser algo como <Un  bosque frondoso lleno de árboles con flores  y ruinas> o <Una posada antigua llena de gente y un  ambiente amigable> Es decir, dado el contexto tienes que generar una descripción, MUY CORTA, MÁXIMO DE 10 PALABRAS .Asegúrate de que la imagen sea lo más  grande posible, es decir, que esté en 4k\n"
        "Asegurate de que en caso de que en el  contexto anterior el personaje esté hablando con alguna persona de que la descripción generada sea la de esta persona, por ejemplo: <Una muchacha rubia y con aspecto agradable> o <Un hombre mayor pero fornido> HAZ ESTO SOLO SI ESTÁ HABLANDO EXPLÍCITAMENTE CON OTRO PERSONAJE\n"
        "Asegúrate también de que la descripción sea explícitamente del lugar en el que se encuentra actualmente el protagonista de la historia (el jugador)\n"
    )

    validation_messages = [
        {"role": "system", "content": "Eres un generador de texto visual. Tu tarea es proporcionar descripciones muy cortas de la escena actual dado el contexto proporcionado"},
        {"role": "user", "content": validation_prompt}
    ]

    response = openai.ChatCompletion.create(
        model=modeloAUsar,
        messages=validation_messages,
        max_tokens=256,
        temperature=0.7
    )
    validation_response = response.choices[0].message["content"]
    #print(f"\n  Texto para generación de imagen: {validation_response}")
    return validation_response

#Función para traducir  el texto que se va a usar para generar la imagen al inglés ya que fue  entrenado en este idioma el modelo.
def translateToEnglish(text):
    translation_prompt = (
        f"El siguiente texto está en español: {text}\n"
        "Por favor, tradúcelo al inglés de la manera más precisa posible."
    )

    translation_messages = [
        {"role": "system", "content": "Eres un traductor profesional. Tu tarea es traducir cualquier texto del español al inglés de manera precisa."},
        {"role": "user", "content": translation_prompt}
    ]

    response = openai.ChatCompletion.create(
        model=modeloAUsar,
        messages=translation_messages,
        max_tokens=256,
        temperature=0.7
    )


    translation_response = response.choices[0].message["content"]
    #print(f"\n  Traducción al inglés: {translation_response}")
    return translation_response

def generate_image(prompt_text, output_file_name):
    if torch.cuda.is_available():
        device = "cuda"
    else:
        device = "cpu"
        print("Advertencia: CUDA no está disponible, utilizando CPU en su lugar.")

    model_id = 'jeruso/StableDiffusionSteampunk'

    try:
        fine_tuned_model = StableDiffusionPipeline.from_pretrained(
            model_id,
            torch_dtype=torch.float16 if device == "cuda" else torch.float32
        )

        fine_tuned_model = fine_tuned_model.to(device)
        prompt_textEnglish = translateToEnglish(prompt_text)
        prompt_textFinal = prompt_textEnglish.rstrip(".") + " in steampunk style."

        # Generar la imagen basada en el prompt
        #print(f"Generando imagen con el prompt: {prompt_textFinal}")
        image = fine_tuned_model(prompt=prompt_textFinal).images[0]

        # Mostrar la imagen generada con matplotlib
        plt.imshow(image)
        plt.axis("off")
        plt.show()

        image.save(output_file_name)
        #print(f"Imagen guardada en: {output_file_name}")
    except Exception as e:
        print(f"Error al generar la imagen: {e}")


#Función para comprobar si la historia  ha cambiado de escena y es necesario generar una nueva imagen.
def check_scene_change(last_context, new_context):
    scene_change_prompt = (
        f"El protagonista estaba en el siguiente contexto:\n"
        f"Contexto anterior: {last_context}\n\n"
        f"Ahora el protagonista está en el siguiente contexto:\n"
        f"Nuevo contexto: {new_context}\n\n"
        f"Considera que un cambio de escenario ocurre si el entorno o lugar físico donde está el protagonista "
        f"ha cambiado significativamente. Esto incluye pasar de un lugar a otro (por ejemplo, de un bosque a un castillo), "
        f"o de una situación que implica un cambio de entorno significativo (como estar en una cueva y luego en un mercado). "
        f"Sin embargo, si solo han cambiado pequeños detalles dentro del mismo entorno (por ejemplo, cambiar de habitación dentro del mismo edificio o "
        f"hablar con una persona en el mismo lugar), NO consideres que el escenario ha cambiado.\n\n"
        f"¿Ha cambiado explícitamente el escenario del protagonista entre el contexto anterior y el nuevo contexto? "
        f"Responde solo con 'sí' o 'no'."
    )

    scene_change_messages = [
        {"role": "system", "content": "Eres un evaluador experto en determinar si ha habido un cambio significativo de escenario en una narrativa."},
        {"role": "user", "content": scene_change_prompt}
    ]

    response = openai.ChatCompletion.create(
        model=modeloAUsar,
        messages=scene_change_messages,
        max_tokens=64,
        temperature=0.5
    )

    scene_change_response = response.choices[0].message["content"].strip().lower()

    return "sí" in scene_change_response or "si" in scene_change_response

#Función para un correcto formato en la salida  en coolab.
def format_text_preserving_structure(text, max_length):
    lines = text.splitlines()
    formatted_text = ""

    for line in lines:
        if line.strip().startswith(tuple([f"{i}." for i in range(10)])):
            formatted_text += line + "\n"
        elif len(line) > max_length:
            words = line.split()
            current_line = ""
            for word in words:
                if len(current_line) + len(word) + 1 > max_length:
                    formatted_text += current_line + "\n"
                    current_line = word
                else:
                    if current_line:
                        current_line += " " + word
                    else:
                        current_line = word
            formatted_text += current_line + "\n"
        else:
            formatted_text += line + "\n"

    return formatted_text

# Función para actualizar el historial de la conversación y validar la acción
def interact(messages, user_input):
    global numImage
    context = "".join([f"{msg['role']}: {msg['content']}\n" for msg in messages])
    last_context = context
    while True:
        valid = validate_action(user_input, context, inventory, cartera)
        if valid:
            messages.append({"role": "user", "content": user_input})
            response_text = generate_response(messages)

            if check_scene_change(last_context, response_text):
                print("El escenario ha cambiado.Se generará una nueva imagen.")
                imageTextGeneration = textToImageGeneration(response_text)
                output_filename = "imagen" + str(numImage) + ".png"
                generate_image(prompt_text=imageTextGeneration, output_file_name=output_filename)
                numImage += 1
            else:
                print("El escenario no ha cambiado. No se generará una nueva imagen.")

            update_inventory(response_text)
            success = update_wallet(response_text)
            if not success:
                response_text += "\nNo tienes suficientes fondos para realizar esta acción."
            response = {"role": "assistant", "content": response_text}
            messages.append(response)
            return response, messages
        else:
            print("Esa acción no tiene sentido en el contexto de la historia, el inventario o la cartera. Por favor, elige otra acción.")
            user_input = input("Tú: ")

def print_inventory(inventory):
    if not inventory:
        print("Inventario actual: Vacío")
    else:
        print("Inventario actual:")
        for item, cantidad in inventory.items():
            print(f"  - {item}: {cantidad}")

def print_wallet(cartera):
    if not cartera:
        print("Cartera actual: Vacía \n")
    else:
        print("Cartera actual:")
        for currency, amount in cartera.items():
            print(f"  - {currency}: {amount} \n")

# Bucle principal de interacción con prompting
print("¡Bienvenido a la aventura! Escribe tus acciones a continuación (escribe 'salir' para terminar):")
first_interaction = True
while True:
    user_input = input("Tú: ")
    if user_input.lower() == "salir":
        break
    if first_interaction:
        update_inventory(user_input)
        update_wallet(user_input)
        first_interaction = False
    # Añadir prompting a la entrada del jugador
    response, messages = interact(messages, user_input)
    formatted_response = format_text_preserving_structure(response['content'], max_length=120)
    print(formatted_response)
    print_inventory(inventory)
    print_wallet(cartera)


True
¡Bienvenido a la aventura! Escribe tus acciones a continuación (escribe 'salir' para terminar):
Tú: Soy el  caballero susi, cojo una espada  y un escudo y quiero vivir  aventuras  en el  mundo mágico de Azinoz.
Esa acción no tiene sentido en el contexto de la historia, el inventario o la cartera. Por favor, elige otra acción.
Tú: Soy el  caballero susi, cojo una espada  y un escudo y quiero vivir  aventuras  en el  mundo mágico de Azinoz.
Esa acción no tiene sentido en el contexto de la historia, el inventario o la cartera. Por favor, elige otra acción.
Tú: Soy el  caballero susi y quiero vivir  aventuras  en el  mundo mágico de Azinoz.
El escenario ha cambiado.Se generará una nueva imagen.


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model_index.json:   0%|          | 0.00/717 [00:00<?, ?B/s]

Fetching 16 files:   0%|          | 0/16 [00:00<?, ?it/s]

(…)ature_extractor/preprocessor_config.json:   0%|          | 0.00/493 [00:00<?, ?B/s]

...

unet/config.json:   0%|          | 0.00/1.90k [00:00<?, ?B/s]

tokenizer/vocab.json:   0%|          | 0.00/1.11M [00:00<?, ?B/s]

vae/config.json:   0%|          | 0.00/970 [00:00<?, ?B/s]

diffusion_pytorch_model.safetensors:   0%|          | 0.00/335M [00:00<?, ?B/s]

KeyboardInterrupt: 